# Automated pipeline for the extraction and analysis of flowcam data with special attention paid to the detection and characterization of *Phaeocystis antarctica*. 

## January 2018
## McMurdo Station, Antarctica

### Harriet Alexander

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy import ndimage as ndi
import glob
from itertools import compress
import pandas as pd
import pickle as cpk

from skimage import measure
from skimage import io
from skimage import color
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage import segmentation 
from skimage import data
from skimage import measure
from skimage import filters
from skimage import morphology
from skimage import restoration
from skimage.filters import try_all_threshold
from skimage import util
from skimage import draw


%matplotlib inline

# Cutting individual tifs from mother image

In [2]:
def rgb_to_bw_background(I, thresh=None):
    '''
    rgb_to_bw_background: converts image to bw based on chosen threshold; automatically use yen. 

    I = image handle (intialized with scikit image package using io.imread)
    thresh = threhold value (float)
    '''
    Ig = color.rgb2gray(I)
    if thresh is None:
        thresh = filters.threshold_yen(Ig)
    Ibw = Ig > thresh
    Ibw = Ibw.astype(int)
    
    return(Ibw)

def cutImage(motherTif, outdir, pad=10,):
    '''
    cutImage: takes in FlowCam collage and cuts out individual
    images from the background and saves them as tifs

    motherTif = filename of collage tif file (str)
    pad = size of padding to put around the image (int)
    '''
    # read in image
    motherimg = io.imread(motherTif)
    
    # get bw verion of image
    imgbw = rgb_to_bw_background(motherimg, 0)
    
    # pad with black
    imgbw = np.pad(imgbw, [pad,pad], 'constant', constant_values=0)

    # label regions
    imlab=measure.label(imgbw)
    
    # get boudning box
    rp = measure.regionprops(imlab)
    
    
    # get name for tif file
    mother_out = motherTif.split('/')[-1].split('.')[0]
    
    # make directory if doesn't exist
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    
    # cut images
    for i,r in enumerate(rp):
        bbox =r['bbox']
        # bbox = (min_row, min_col, max_row, max_col)
        x1 = bbox[0]-pad
        x2 = bbox[2]-pad
        y1 = bbox[1]-pad
        y2 = bbox[3]-pad
        outimg = motherimg[x1:x2, y1:y2, :]
        np.shape(outimg)
        io.imsave(outdir + '/' + mother_out+'_' + str(i).zfill(4)+'.tif', outimg)
        
def batch_cut_images(directory, outdirectory, ftype = '*tif', test=False):
    '''
    batch_cut_images: loops through a directory and runs cutImage on 
    all *tif files in the directory

    directory = name of directory (str)
    outdirectory = name of the directory to save the new imgages to (str)
    ftype = file type to search for (str)
    '''
    for ifile in glob.glob('/'.join([directory, ftype])):
        dirName = ifile.split('/')[-2] 
        name = ifile.split('/')[-1] 
        
        # skip binary and calibration files generated by the flow cam
        if name.endswith('bin.tif'):
            pass
        elif name.startswith('cal_image'):
            pass
        else:
            if test ==True:
                print(directory+name)
            else:   
                cutImage(directory + name, '/'.join([outdirectory, dirName]))

            

## Loop over raw data files in `data/rawdata/`, generate new individual tif files, and save to `data/processed-data/individual-tifs`

In [3]:
for i, ifolder in enumerate(glob.glob('test-data/*/')):
    outdir = 'test-data/processed-data/individual-tifs/'+ifolder.split('/')[-2]
    if os.path.exists(outdir):
        pass
    else:
        print(ifolder)
        batch_cut_images(directory=ifolder, outdirectory= 'test-data/processed-data/individual-tifs/')


data/rawdata/20180129-IceEdge-sample0127-0m-500ml/
data/rawdata/20180129-IceEdge-sample0127-10m-200ml/
data/rawdata/20180129-IceEdge-sample0127-10m-50ml/
data/rawdata/20180129-IceEdge-sample0127-nettow-20ml/


/Users/halexand/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: data/processed-data/individual-tifs//20180129-IceEdge-sample0127-nettow-20ml/20180129-IceEdge-sample0127-nettow-20ml_000076_0011.tif is a low contrast image
  warn('%s is a low contrast image' % fname)


# Processing of individual tifs to find edges

In [4]:
def rbg_to_bw(I, yen=True):
    '''
    rgb_to_bw: converts image to bw based on chosen threshold; automatically use yen. 

    I = image handle (intialized with scikit image package using io.imread)
    yen = T/F use yen method for threshold or otsu (bool)
    '''
    # convert to gray
    Ig = color.rgb2gray(I)
    
    #convert to binary
    if yen:
        thresh = filters.threshold_yen(Ig)
    else:
        thresh = filters.threshold_otsu(Ig)
    Ibw = (Ig <= thresh)
    Ibw = Ibw.astype(int)
    
    return(Ibw)

def edge_detect(Ibw, sobel=True):
    '''
    edge_detect: run edge detection on a provided binary image

    Ibw = image handle for binary image
    sobel = TRUE = use sobel method for edge detection; FALSE = use canny method (bool)
    '''
    
    if sobel:
        edges = filters.sobel(Ibw)
    else:
        edges = feature.canny(Ibw)
        
    return edges

def dilation(edges, selem_size=8):
    '''
    dilation: run dilation on set of edges with a specified selem_size for a disk
    
    edges = image handle for edges file
    selem_size = size of selem to use for the dilation (int)
    '''
    
    selem = morphology.disk(selem_size)
    dilated = morphology.binary_dilation(edges, selem)
    return dilated

def fill_erode_close(dilated, selem_size=8):
    '''
    fill_eroted_close: runclosing and reconstruction on provided binary image with selected selem_siz
    
    dilated = image handle for dilated or processed binary image file
    selem_size = size of selem to use for the dilation (int)
    '''
    # fill the edges
    seed = np.copy(dilated)
    seed[1:-1, 1:-1] = dilated.max()
    mask = dilated
    filled = morphology.reconstruction(seed, mask, method='erosion')
    
    # close small holes
    selem = morphology.disk(selem_size)
    close_filled = morphology.binary_closing(filled, selem)
    close_filled = close_filled.astype('int')

    return close_filled

def manual_fill_holes(close_filled):
    '''
    manual_fill_holes: manually identify small holes within the image region and draw a circle 
    around them equal to the diameter of the longest axis or equivalent diameter
    
    close_filled = image handle for binary image with holes to be filled. Holes = 0s. 
    '''
    
    # label the inverse of the close_filled object
    close_filled_label = measure.label(1-close_filled, connectivity=2)
    shape = np.shape(close_filled_label)
    
    #get regional properties
    rp_cfl= measure.regionprops(close_filled_label)
    a=[]
    a_lab=[]
    for i in rp_cfl:
        a.append(i.area)
        a_lab.append(i.label)
    asum=sum(a)
    a_lab_sorted = [x for _,x in sorted(zip(a,a_lab), reverse=True)]
    rp_cfl.pop(a_lab_sorted[0]-1)
    # cut based on size instead of order?
    # for aa in a:
    #     if aa > 0.2 * asum:
    #         print(aa)
    
    # draw circles over remaining holes
    base = close_filled.copy()
    
    for r in rp_cfl:
        radius =  r.major_axis_length/2
        center = r.centroid
        
        # make sure circle doesn't exceed the size of the image
        if np.any((np.max(radius) + center> shape[0])|(np.max(radius) + center> shape[1])):
            radius =  r.equivalent_diameter/2
        if np.any((np.max(radius) + center> shape[0])|(np.max(radius) + center> shape[1])):
            pass
        else:
            rr, cc = draw.circle(center[0], center[1], radius)
            base[rr, cc] = 1
            
    return base

def get_final_edge(Ifinal):
    '''
    get_final_edge: take in the final binary image and detect the edges. return bool of edges. 
    
    Ifinal = image handle for final binary file of total region
    '''
    edge = edge_detect(Ifinal)
    edge[edge==0] = np.nan
    edge[edge>0]=1
    
    return(edge)



# Batch process individual tif files to identify edges 

In [5]:

def identify_edges(I, plot = False):
    '''
    identify_edges: pipeline to run analysis on provided image handle 
    
    I = image handle for rgb input tif file
    plot = TRUE = plot all the figures and save as edge file. Adds a lot of time. 
    '''
    Ibw = rbg_to_bw(I)
    Iedge = edge_detect(Ibw)
    Idilation = dilation(Iedge)
    Ifilled = fill_erode_close(Idilation)
    Imanfill = manual_fill_holes(Ifilled)
    Ifinal = fill_erode_close(Imanfill)
    Iedge_final = get_final_edge(Ifinal)
    
    if plot == True:
        fig, ax = plt.subplots(1,4, figsize = [12,12])
        ax[0].imshow(Ibw, cmap=plt.cm.gray)
        ax[1].imshow(Idilation, cmap=plt.cm.gray)
        ax[2].imshow(Ifinal, cmap=plt.cm.gray)
        ax[3].imshow(I)
        ax[3].imshow(Iedge_final, cmap=plt.cm.hs)
        for a in ax:
            a.set_xticklabels('')
            a.set_yticklabels('')

        return(Iedge_final, Ifinal, fig, ax)
    
    else: 
        return(Iedge_final, Ifinal, Ibw)
    
    
def batch_process_images(wd, ftype = '*tif', makeplots = False):
    '''
    batch_process_images: batch process directories of files. Find tif files nad process them. 
    
    I = image handle for rgb input tif file
    plot = TRUE = plot all the figures and save as edge file. Adds a lot of time. 
    
    returns a dictionary that is indexed by the file name and contains all the different images. 
    '''
    c=0
    Idict = {}
    for ifile in glob.glob(wd + ftype):
        name = ifile.split('/')[-1] 
        # read in image
        try:
            I = io.imread(ifile)
        except: 
            print('Could not read ' + ifile)
            continue
        # if you want to save plots
        if makeplots == True:
            (Iedge_final, Ifinal, fig, ax) = identify_edges(I, plot = makeplots)
            outfile = ifile[:-4]+'_edge.tif'
            fig.savefig(outfile)
            plt.close()
        else:
            (Iedge_final, Ifinal, Ibw) = identify_edges(I, plot = makeplots)

        #output dictionary: rgb = rgb image, edge = final edge file, 
        #binary = final binary of larger regions, Ibw = binary of original image
        Idict[name]={'rbg':I, 'edge':Iedge_final, 'binary': Ifinal, 'Ibw': Ibw}

#         if c ==5:
#             break
#         if c%100 ==0:
#             print(c)
        c+=1

    return Idict
            

## Process experimental data in `data/processed-data/indvidual-tifs/`

In [8]:
# masterDict = {}

pickleDir = 'test-data/processed-data/pickle-dicts/'
if not os.path.exists(pickleDir):
    os.makedirs(pickleDir)

for i, folder in enumerate(glob.glob('test-data/processed-data/individual-tifs/*/')):
    outName = (folder.split('/')[-2])
    pout = pickleDir + outName + '.pickle'
    print(outName)
    if os.path.exists(pout):
        pass
    else:
        Idict = batch_process_images(folder, makeplots = False)
#         masterDict[outName]=Idict
        outPickle = open(pickleDir + outName+'.pickle', 'wb')
        cpk.dump(Idict, outPickle)
        outPickle.close()
#         if i == 1:
#             break

20180129-IceEdge-sample0127-0m-500ml


/Users/halexand/anaconda3/lib/python3.6/site-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from int64 to float64
  .format(dtypeobj_in, dtypeobj_out))
/Users/halexand/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:121: RuntimeWarning: invalid value encountered in greater


20180129-IceEdge-sample0127-10m-200ml
20180129-IceEdge-sample0127-10m-50ml
20180129-IceEdge-sample0127-nettow-20ml


In [9]:
for ff in ['test-data/test-set/', 'test-data/training-set/']:
    for i, folder in enumerate(glob.glob(ff+'*/')):
        print(folder)
        outName = folder.split('/')[-2]
        outName = outName + folder.split('/')[-3]
        pout = pickleDir + outName + '.pickle'
        if os.path.exists(pout):
            pass
        else:
            Idict = batch_process_images(folder, makeplots = False)
#             training_dict[outName]=Idict
            outPickle = open(pickleDir + outName+'.pickle', 'wb')
            cpk.dump(Idict, outPickle)
#     if i == 1:
#         break

data/test-set/notPhaeo/
data/test-set/Phaeo/
data/training-set/notPhaeo/
data/training-set/Phaeo/


# Calculation of regional properties for each individual tif image

In [9]:
def get_largest_region(rbg, Ibw, Ifinal):
    '''
    get_largest_region: gets the region with the largest area identified in Ifinal 
    
    rbg = image handle for rgb input tif file
    Ibw = image handle for raw binary image
    Ifinal = image handle for the dilated binary image (final)
    
    Returns  binary image (Ibw_regional) and color image (rbg_regional) masked by the largest region (max_binary)
    '''
    
    # label the binary image
    lab_bin = measure.label(Ifinal, connectivity = 2)
    rp = measure.regionprops(lab_bin)
    
    # get the label with the max area
    areas = [r.area for r in rp]
    labs = [r.label for r in rp]
    Ibw_regional = Ibw.copy()
    
    # get the largest area
    if len(areas)>0:
        max_lab = labs[areas.index(max(areas))]
        # get bool for the max area and the not max area
        not_max_binary = lab_bin != max_lab
        max_binary = lab_bin == max_lab
        # set things outside of largest area to 0 in binary and color image
        Ibw_regional[not_max_binary] = 0
        rbg_regional = np.where(max_binary[...,None], rbg, 0)
    else: 
        # if no regions present
        Ibw_regional = -9999999
        rbg_regional = -9999999
        max_binary = -9999999
        pass
    
    return Ibw_regional, rbg_regional, max_binary


def number_sub_areas(Ibw_regional):
    '''
    number_sub_areas: gets the number of subregions in the region with the largest area
    
    Ibw_regional = image handle binary original image masked by the largest region
    
    Returns number of sub regions (int)
    '''

    # calculate the number of distinct sub areas within the blob of the largest region
    Ibw_reg_lab = measure.label(Ibw_regional)
    rp = measure.regionprops(Ibw_reg_lab)
    num_regions = len(rp)
    return num_regions

def proportion_occupied(Ibw_regional, max_binary):
    '''
    proportion_occupied: calculates the proprotion of the largest region that is 
    occupied by sub regions
    
    Ibw_regional = image handle binary original image masked by the largest region
    max_binary = image handle for binary image of the largest region
    
    Returns proprotion of occupied area (float) 
    '''

    # calualte the proportion of the largest region occupied by subregions (i.e. the density of the subregions)
    reginonal_area = np.sum(Ibw_regional)
    max_area = np.sum(max_binary)
    prop_oc = reginonal_area/ float(max_area)
    return prop_oc 
    
    
def mean_size_area(Ibw_regional, cutoff = 50):
    '''
    mean_size_area: calculates the mean and std area of the subareas within the largest region 
    above a specified pixel size cutoff value 
    
    Ibw_regional = image handle binary original image masked by the largest region
    cutoff = number of pixels required for inclusion of a sub area in the analysis (int)
    
    Returns area distribution (areas), mean area (a_mean), and standard dev (a_std)
    '''

    # calculate the mean area for the sub regions of the largest region with a specified size cut off value
    Ibw_reg_lab = measure.label(Ibw_regional)
    rp = measure.regionprops(Ibw_reg_lab)
    areas = []
    for r in rp: 
        a = r.area
        lab = r.label
        if a > cutoff:
            areas.append(a)
    a_mean  = np.mean(areas)
    a_std = np.std(areas)
    
    return areas, a_mean, a_std

def get_sub_region_color(rbg_regional):
    '''
    get_sub_region_color: returns the mean and std color intensity for the subregions 
    in the rgb_regional image across R, G, and B. 
    
    rbg_regional = image handle rgb original image masked by the largest region
    
    Returns dictionary containing the mean, stdev, median, color_entropy for 
    each of the color channels of occupied areas as well as the ratios between colors 
    '''
    
    # loop over color channels
    rgb_dict ={}
    ratio_dict ={}
    for i, cc in enumerate(['red', 'green', 'blue']):
        color_chan = pd.DataFrame(rbg_regional[:, :, i])
        color_chan[color_chan == 0] = np.nan
        c_mean = color_chan.mean().mean()
        c_std = color_chan.std().std()
        c_median = color_chan.median().median()
        a_chan=np.array(color_chan.fillna(0))
        shan_ent = measure.shannon_entropy(a_chan)
        in_dict = {'color_mean': c_mean, 'color_std': c_std, 
                   'color_median': c_median, 'color_shannon_entropy': shan_ent}
        rgb_dict[cc]=in_dict
    ratio_dict['ratio_r_b']= rgb_dict['red']['color_mean']/rgb_dict['blue']['color_mean']
    ratio_dict['ratio_r_g']= rgb_dict['red']['color_mean']/rgb_dict['green']['color_mean']
    ratio_dict['ratio_b_g']= rgb_dict['blue']['color_mean']/rgb_dict['green']['color_mean']

    return rgb_dict, ratio_dict



def regionProps(Idict):
    '''
    regionProps: the built in region props (listed below) for the largest region 
    
    Idict = dictionary containing image information
    
    Adds to the input dictionary Idict and returns a pandas df with the property values
    '''
    props_included = ['area','bbox_area','convex_area','eccentricity','equivalent_diameter',
                     'extent','filled_area','major_axis_length','minor_axis_length', 
                      'orientation', 'perimeter','solidity','shannon_entropy']
    out_pd = pd.DataFrame(columns=props_included)

    for i,I in enumerate(Idict.keys()):
        # load in data
        # the largest area to be considered
        Ifinal = Idict[I]['binary']
        mask = Ifinal.astype('bool')
        # color image
        rbg = Idict[I]['rbg']
        # bw rendering of the color image
        Ibw = Idict[I]['Ibw']

        # get the regional Ibw region, rbg region, and max_binary image of the largest region
        Ibw_regional, rbg_regional, max_binary = get_largest_region(rbg, Ibw, Ifinal)
        if np.any(Ibw_regional ==  -9999999):
            pass
        else:
            max_binary = max_binary.astype(int)
        
            # calcualte region props for the max binary region
            rp = measure.regionprops(max_binary)
            r = rp[0]
            for rr in r:
                if rr in props_included:
                    # add to df
                    out_pd.loc[I, rr] = r[rr]
         
            Idict[I]['Ibw_regional'] = Ibw_regional
            Idict[I]['rbg_regional'] = rbg_regional
            Idict[I]['max_binary'] = max_binary

            out_pd.loc[I, 'shannon_entropy'] = measure.shannon_entropy(rbg_regional)
                
    return out_pd, Idict

def user_defined_properties(Idict, out_pd):
    '''
    user_defined_properties: returns user specified regional properites to pandas dictionary
    
    Idict = dictionary containing image information
    out_pd = pandas df out put by regionProps function
    
    Returns region props pandas dataframe
    '''
    for i,I in enumerate(out_pd.index):

        Ibw_regional = Idict[I]['Ibw_regional']
        rbg_regional = Idict[I]['rbg_regional'] 
        max_binary = Idict[I]['max_binary']

        num_subregions = number_sub_areas(Ibw_regional)
        prop_oc = proportion_occupied(Ibw_regional, max_binary)
        areas, a_mean, a_std = mean_size_area(Ibw_regional)
        rgb_dict, ratio_dict = get_sub_region_color(rbg_regional)
        
        out_pd.loc[I, 'num_subregions'] = num_subregions
        out_pd.loc[I, 'proportion_occupied'] = prop_oc
        out_pd.loc[I, 'subregion_mean_area'] = a_mean
        out_pd.loc[I, 'subregion_std_area'] = a_std
        
        for key in rgb_dict.keys():
            for subkey in rgb_dict[key].keys():
                out_pd.loc[I, key+'_'+subkey] = rgb_dict[key][subkey]
        for key in ratio_dict.keys():
            out_pd.loc[I, key] = ratio_dict[key]
            
    return out_pd
  
def properties_to_csv(Idict, filename): 
    out_pd, Idict = regionProps(Idict)
    out_pd = user_defined_properties(Idict, out_pd)
    out_pd.to_csv(filename)
    return(Idict, out_pd)



In [10]:
csvdir = 'test-data/processed-data/region_props_csv/'
rp_dict = {}
if not os.path.exists(csvdir):
    os.mkdir(csvdir)

for pfile in glob.glob('test-data/processed-data/pickle-dicts/*pickle'):
    print(pfile)
    name = pfile.split('/')[-1].split('.')[0]
    csvfile = name +'.csv'
    if os.path.exists(csvdir+csvfile):
        pass
    else:
        Idict = cpk.load(open(pfile, 'rb'))
        Idict, outpd = properties_to_csv(Idict, csvdir + csvfile)
        cpk.dump(Idict, open(pfile+'2', 'wb'))
        rp_dict[name] = outpd

data/processed-data/pickle-dicts/20180127-PhaeoEx-AB12-1-T7-200ml.pickle
data/processed-data/pickle-dicts/20180127-PhaeoEx-AB12-2-T7-200ml.pickle
data/processed-data/pickle-dicts/20180127-PhaeoEx-AB12-3-T7-200ml.pickle
data/processed-data/pickle-dicts/20180127-PhaeoEx-Anti-1-T7-200ml.pickle
data/processed-data/pickle-dicts/20180129-IceEdge-sample0127-0m-500ml.pickle


/Users/halexand/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/halexand/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/halexand/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Users/halexand/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Users/halexand/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


data/processed-data/pickle-dicts/20180129-IceEdge-sample0127-10m-200ml.pickle
data/processed-data/pickle-dicts/20180129-IceEdge-sample0127-10m-50ml.pickle
data/processed-data/pickle-dicts/20180129-IceEdge-sample0127-nettow-20ml.pickle
